# Evaluate RAG with LlamaIndex Locally

> Notes:
> 1. I will use LM Studio to replace the OpenAPI API.
> 2. The LLM Model is mistral-7b-instruct-v0.2.Q6_K

In this notebook we will look into building an RAG pipeline and evaluating it with LlamaIndex. It has following 3 sections.

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG with LlamaIndex.
3. Evaluating RAG with LlamaIndex.

**Retrieval Augmented Generation (RAG)**

LLMs are trained on vast datasets, but these will not include your specific data. Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data during the generation process. This is done not by altering the training data of LLMs, but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and and prepared for queries or “indexed”. User queries act on the index, which filters your data down to the most relevant context. This context and your query then go to the LLM along with a prompt, and the LLM provides a response.

Even if what you’re building is a chatbot or an agent, you’ll want to know RAG techniques for getting data into your application.

![RAG Overview](./images/llamaindex_rag_overview.png)

**Stages within RAG**

There are five key stages within RAG, which in turn will be a part of any larger application you build. These are:

**Loading:** this refers to getting your data from where it lives – whether it’s text files, PDFs, another website, a database, or an API – into your pipeline. LlamaHub provides hundreds of connectors to choose from.

**Indexing:** this means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.

**Storing:** Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

**Querying:** for any given indexing strategy there are many ways you can utilize LLMs and LlamaIndex data structures to query, including sub-queries, multi-step queries and hybrid strategies.

**Evaluation:** a critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures of how accurate, faithful and fast your responses to queries are.

## Build RAG system.

Now that we have understood the significance of RAG system, let's build a simple RAG pipeline.

In [ ]:
%pip install llama-index

In [ ]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

from llama_index.evaluation import generate_question_context_pairs
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.evaluation import generate_question_context_pairs
from llama_index.evaluation import RetrieverEvaluator
from llama_index.llms import OpenAI
from llama_index.embeddings import resolve_embed_model

import os
import pandas as pd

#### Load Data and Build Index.

In [117]:
# load data from data directory
documents = SimpleDirectoryReader("data").load_data()

# bge-m3 embedding model
# https://huggingface.co/BAAI/bge-base-en-v1.5/tree/main
embed_model = resolve_embed_model("local:BAAI/bge-base-en-v1.5")

# Load LM Studio LLM model
llm = OpenAI(api_base="http://localhost:1234/v1", api_key="not-needed")

# Index the data
service_context = ServiceContext.from_defaults(
    embed_model=embed_model, llm=llm,
)

# Transform data to Nodes struct
node_parser = SimpleNodeParser.from_defaults(chunk_size=512, chunk_overlap=128)
nodes = node_parser.get_nodes_from_documents(documents)

# vetorize
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

Build a QueryEngine and start querying.

In [118]:
query_engine = vector_index.as_query_engine()

In [122]:
response_vector = query_engine.query("What did the author do growing up?")

Check response.

In [123]:
response_vector.response

"The author grew up in New Hampshire and spent most of his time reading science fiction and painting. He attended a boarding school in Massachusetts for high school, where he continued to paint and read. After graduating from college with a degree in computer science, he worked at various software companies before starting his own company, Viaweb, which was sold to Yahoo in 1996. He then moved to California and tried to focus on painting, but found it difficult due to lack of energy and motivation. He eventually returned to New York and resumed painting, this time with more success.\n### Explanation:\nThe author's childhood was marked by a love for reading science fiction and painting. He attended a boarding school in Massachusetts, where he continued to pursue these interests. After college, he worked in the software industry before starting his own company, Viaweb, which was sold to Yahoo in 1996. Following the sale of Viaweb, the author moved to California with the intention of focu

By default it retrieves `two` similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [124]:
# First retrieved node
response_vector.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [125]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"Nor had I changed my grad student lifestyle significantly since we started. So when Yahoo bought us it felt like going from rags to riches. Since we were going to California, I bought a car, a yellow 1998 VW GTI. I remember thinking that its leather seats alone were by far the most luxurious thing I owned.\n\nThe next year, from the summer of 1998 to the summer of 1999, must have been the least productive of my life. I didn't realize it at the time, but I was worn out from the effort and stress of running Viaweb. For a while after I got to California I tried to continue my usual m.o. of programming till 3 in the morning, but fatigue combined with Yahoo's prematurely aged culture and grim cube farm in Santa Clara gradually dragged me down. After a few months it felt disconcertingly like working at Interleaf.\n\nYahoo had given us a lot of options when they bought us. At the time I thought Yahoo was so overvalued that they'd never be worth anything, but to my astonishment the stock went

We have built a RAG pipeline and now need to evaluate its performance. We can assess our RAG system/query engine using LlamaIndex's core evaluation modules. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and a range of queries.

While it's beneficial to examine individual queries and responses at the start, this approach may become impractical as the volume of edge cases and failures increases. Instead, it may be more effective to establish a suite of summary metrics or automated evaluations. These tools can provide insights into overall system performance and indicate specific areas that may require closer scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

*   **Retrieval Evaluation:** This assesses the accuracy and relevance of the information retrieved by the system.
*   **Response Evaluation:** This measures the quality and appropriateness of the responses generated by the system based on the retrieved information.

#### Question-Context Pair Generation:

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response. `LlamaIndex` offers a `generate_question_context_pairs` module specifically for crafting questions and context pairs which can be used in the assessment of the RAG system of both Retrieval and Response Evaluation. For more details on Question Generation, please refer to the [documentation](https://docs.llamaindex.ai/en/stable/examples/evaluation/QuestionGeneration.html).

In [126]:
qa_dataset = generate_question_context_pairs(
    nodes,
    llm=llm,
    num_questions_per_chunk=20
)

100%|██████████| 46/46 [03:53<00:00,  5.08s/it]


#### Retrieval Evaluation:

We are now prepared to conduct our retrieval evaluations. We will execute our `RetrieverEvaluator` using the evaluation dataset we have generated.

We first create the `Retriever` and then define two functions: `get_eval_results`, which operates our retriever on the dataset, and `display_results`, which presents the outcomes of the evaluation.

Let's create the retriever.

In [127]:
retriever = vector_index.as_retriever(similarity_top_k=2)

Define `RetrieverEvaluator`. We use **Hit Rate** and **MRR** metrics to evaluate our Retriever.

**Hit Rate:**

Hit rate calculates the fraction of queries where the correct answer is found within the top-k retrieved documents. In simpler terms, it’s about how often our system gets it right within the top few guesses.

**Mean Reciprocal Rank (MRR):**

For each query, MRR evaluates the system’s accuracy by looking at the rank of the highest-placed relevant document. Specifically, it’s the average of the reciprocals of these ranks across all the queries. So, if the first relevant document is the top result, the reciprocal rank is 1; if it’s second, the reciprocal rank is 1/2, and so on.

Let's check these metrics to check the performance of out retriever.

In [128]:
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

In [ ]:
# Evaluate
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

Let's define a function to display the Retrieval evaluation results in table format.

In [131]:
def display_results(name, eval_results):
    """Display results from evaluate."""

    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    hit_rate = full_df["hit_rate"].mean()
    mrr = full_df["mrr"].mean()

    metric_df = pd.DataFrame(
        {"Retriever Name": [name], "Hit Rate": [hit_rate], "MRR": [mrr]}
    )

    return metric_df

In [132]:
display_results("bge-m3 Embedding Retriever", eval_results)

,Retriever Name,Hit Rate,MRR
0,bge-m3 Embedding Retriever,0.0,0.0


#### Observation:

The Retriever with OpenAI Embedding  demonstrates a performance with a hit rate of `0.7586`, while the MRR, at `0.6206`, suggests there's room for improvement in ensuring the most relevant results appear at the top. The observation that MRR is less than the hit rate indicates that the top-ranking results aren't always the most relevant. Enhancing MRR could involve the use of rerankers, which refine the order of retrieved documents. For a deeper understanding of how rerankers can optimize retrieval metrics, refer to the detailed discussion in our [blog post](https://blog.llamaindex.ai/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83).

#### Response Evaluation:

1. FaithfulnessEvaluator: Measures if the response from a query engine matches any source nodes which is useful for measuring if the response is hallucinated.
2. Relevancy Evaluator: Measures if the response + source nodes match the query.

In [134]:
# Get the list of queries from the above created dataset
queries = list(qa_dataset.queries.values())

#### Faithfulness Evaluator

Let's start with FaithfulnessEvaluator.

I will use `mistral-7b-instruct-v0.2.Q6_K` for generating response for a given query and evaluation.

In [135]:
vector_index = VectorStoreIndex(nodes, service_context = service_context)
query_engine = vector_index.as_query_engine()

Create a  FaithfulnessEvaluator.




In [136]:
from llama_index.evaluation import FaithfulnessEvaluator
faithfulness_raven13b = FaithfulnessEvaluator(service_context=service_context)

Let's evaluate on one question.

In [148]:
eval_query = queries[3]
eval_query

'Where did the writer have permission to use the IBM 1401 computer system?'

Generate response first and use faithfull evaluator.

In [149]:
response_vector = query_engine.query(eval_query)

In [150]:
# Compute faithfulness evaluation
eval_result = faithfulness_raven13b.evaluate_response(response=response_vector)

In [151]:
# You can check passing parameter in eval_result if it passed the evaluation.
eval_result.passing

True

#### Relevancy Evaluator

RelevancyEvaluator is useful to measure if the response and source nodes (retrieved context) match the query. Useful to see if response actually answers the query.

Instantiate `RelevancyEvaluator` for relevancy evaluation with `gpt-4`

In [143]:
from llama_index.evaluation import RelevancyEvaluator

relevancy_raven13b = RelevancyEvaluator(service_context=service_context)

Let's do relevancy evaluation for one of the query.

In [144]:
# Pick a query
query = queries[3]
query

'Where did the writer have permission to use the IBM 1401 computer system?'

In [145]:
# Generate response.
# response_vector has response and source nodes (retrieved context)
response_vector = query_engine.query(query)

# Relevancy evaluation
eval_result = relevancy_raven13b.evaluate_response(
    query=query, response=response_vector
)

In [146]:
# You can check passing parameter in eval_result if it passed the evaluation.
eval_result.passing

True

In [147]:
# You can get the feedback for the evaluation.
eval_result.feedback

'Yes. The context states that the writer and his friend Rich Draves had permission to use the IBM 1401 computer system in the basement of their junior high school. The response is consistent with this information.'

#### Batch Evaluator:

Now that we have done FaithFulness and Relevancy Evaluation independently. LlamaIndex has `BatchEvalRunner` to compute multiple evaluations in batch wise manner.

In [152]:
from llama_index.evaluation import BatchEvalRunner

# Let's pick top 10 queries to do evaluation
batch_eval_queries = queries[:10]

# Initiate BatchEvalRunner to compute FaithFulness and Relevancy Evaluation.
runner = BatchEvalRunner(
    {"faithfulness": faithfulness_raven13b, "relevancy": relevancy_raven13b},
    workers=8,
)

# Compute evaluation
eval_results = await runner.aevaluate_queries(
    query_engine, queries=batch_eval_queries
)

In [153]:
# Let's get faithfulness score
faithfulness_score = sum(result.passing for result in eval_results['faithfulness']) / len(eval_results['faithfulness'])
faithfulness_score

0.8

In [154]:
# Let's get relevancy score
relevancy_score = sum(result.passing for result in eval_results['relevancy']) / len(eval_results['relevancy'])
relevancy_score


0.8

#### Observation:

Faithfulness score of `0.8` signifies that the generated answers contain little hallucinations and are entirely based on retrieved context.

Relevancy score of `0.8` suggests that the answers generated are not consistently aligned with the retrieved context and the queries.

## Conclusion

In this notebook, we have explored how to build and evaluate a RAG pipeline using LlamaIndex, with a specific focus on evaluating the retrieval system and generated responses within the pipeline. 

LlamaIndex offers a variety of other evaluation modules as well, which you can explore further [here](https://docs.llamaindex.ai/en/stable/module_guides/evaluating/root.html)